In [1]:
from typing import TypeVar
import sqlalchemy as sa
from sqlalchemy import NullPool
import pandas as pd

SelfClickHouseConnection = TypeVar("SelfClickHouseConnection", bound="ClickHouseConnection")

class ClickHouseConnection:
    """Класс контекстного менеджера для работы с ClickHouse"""
    
    def __init__(self, host: str, username: str, password: str, port: str = "9000"):
        """
        Параметры:
            host: хост ClickHouse
            username: имя пользователя ClickHouse
            password: пароль пользователя ClickHouse
            port: порт ClickHouse
        """
        
        self.session = None
        self.__host = host
        self.__port = port
        self.__username = username
        self.__password = password
        
    def __get_url(self) -> str:
        """Метод формирует connection string для подключения к ClickHouse"""
        
        url = "clickhouse+native://{username}:{password}@{host}:{port}"
        url = url.format(
            host = self.__host,
            port = self.__port,
            username = self.__username,
            password = self.__password
        )
        return url
        
    def __enter__(self: SelfClickHouseConnection) -> SelfClickHouseConnection:
        engine = sa.create_engine(self.__get_url(), poolclass=NullPool)
        self.session = engine.connect()
        return self
    
    def __exit__(self, exception_type, exception_value, traceback):
        self.session.close()
        
    def read_sql(self, query: str) -> pd.DataFrame:
        """
        Запрос к БД через фреймворк pandas
        
        Параметры:
            query: SQL в формате строки
            
        Возвращает:
            датафрейм с результатами запроса
        """
        
        with self as connector:
            return pd.read_sql(query, con=connector.session)

In [5]:
ch_host = "10.120.1.11"
ch_port = 8123
ch_username = "amarbuliev"
ch_password = "KuUNgVlPQiSN6dRqk"

connection_manager = ClickHouseConnection(host=ch_host, username=ch_username, password=ch_password)

df = connection_manager.read_sql("SELECT * FROM sandbox.ongoing_projects")

In [10]:
df.leaders[0]

"[{'id': 67, 'fio': 'Фимина Ксения Игоревна', 'name': 'Фимина Ксения Игоревна', 'first_name': 'Ксения', 'last_name': 'Фимина', 'middle_name': 'Игоревна', 'email': ['kfimina@hse.ru'], 'telnum': None, 'ownerPrivilege': 1, 'status': [{'id': 1, 'code': 'project_leader', 'name': 'Руководитель проекта'}, {'id': 5, 'code': 'initiator', 'name': 'Инициатор проекта'}], 'role': 'Руководитель проекта, Инициатор проекта'}, {'id': 8, 'fio': 'Белов Александр Владимирович', 'name': 'Белов Александр Владимирович', 'first_name': 'Александр', 'last_name': 'Белов', 'middle_name': 'Владимирович', 'email': ['AVBelov@hse.ru'], 'telnum': None, 'ownerPrivilege': 1, 'status': [{'id': 2, 'code': 'direction_leader', 'name': 'Руководитель направления'}], 'role': 'Руководитель направления'}]"

In [11]:
import ast

def extract_email_prefixes_and_short_names(df, column_name='leaders'):
    """
    Extract email prefixes and short names from DataFrame based on a specific column containing leaders' information.
    """
    result_dict = {}
    for row in df[column_name]:
        leader_list = ast.literal_eval(row)
        for leader in leader_list:
            # Construct short_name
            short_name = f"{leader['last_name']}"
            if leader["middle_name"] and leader["first_name"]:
                short_name = f"{leader['last_name']} {leader['first_name'][0]}.{leader['middle_name'][0]}."
            elif leader["first_name"]:
                short_name = f"{leader['last_name']} {leader['first_name'][0]}."

            # Extract email
            email = leader.get('email')
            if isinstance(email, list) and email:  # If email is a non-empty list
                first_email = email[0]  # Take the first item
            elif isinstance(email, str):  # If email is already a string
                first_email = email
            else:  # If there is no email or it's not in an expected format
                first_email = None
            
            # Process email to get email prefix
            if first_email:
                email_prefix = first_email.split('@')[0]
            else:  # Handle cases with no valid email
                email_prefix = None
            
            # Update result dictionary with a tuple containing the email prefix and the short name
            result_dict[leader['id']] = (email_prefix, short_name)

    return result_dict

In [12]:
extract_email_prefixes_and_short_names(df)

{67: ('kfimina', 'Фимина К.И.'),
 8: ('AVBelov', 'Белов А.В.'),
 633: ('ameliseenko', 'Елисеенко А.М.'),
 33: ('dkorolev', 'Королев Д.А.'),
 0: (None, ''),
 55: ('prybakov', 'Рыбаков П.В.'),
 1734: ('mvemelyanenko', 'Емельяненко М.В.'),
 62: ('sslastnikov', 'Сластников С.А.'),
 4089: ('va.shin', 'Шин В.А.'),
 7: ('vbashun', 'Башун В.В.'),
 2305: ('daabramov_2', 'Абрамов Д.А.'),
 1980: ('isemichasnov', 'Семичаснов И.В.'),
 1385: ('mvmoiseev', 'Моисеев М.В.'),
 1052: ('aamerikanov', 'Американов А.А.'),
 53: ('a.romanov', 'Романов А.Ю.'),
 51: ('npugach', 'Пугач Н.Г.'),
 676: ('adpiskunov', 'Пискунов А.Д.'),
 749: ('amzayakina', 'Пискунова А.М.'),
 631: ('asyuldashov', 'Юлдашов А.С.'),
 626: ('vburov', 'Буров В.В.'),
 52: ('arolich', 'Ролич А.Ю.'),
 32: ('ikarpova', 'Карпова И.П.'),
 63: ('vstarykh', 'Старых В.А.'),
 30: ('i.ivanov', 'Иванов И.А.'),
 41: ('vminchenkov', 'Минченков В.О.'),
 60: ('avsergeev', 'Сергеев А.В.'),
 1884: ('adkasatkin', 'Касаткин А.Д.'),
 1899: ('vmikolaenko', 'М